In [4]:
import numpy as np
import os
import pandas as pd


Link to Dataset: https://www.kaggle.com/datasets/uciml/glass

In hindsight, the dataset I picked was too small and this didn't help with accuracy and training. The dataset was also skewed

In [1550]:
df = pd.read_csv("glass.csv")

In [1551]:
def get_batches(data,batch_size,bool1 = True):
    x_train = []
    y_train = []
    for i in range(0, len(data), batch_size):
        chunk = (data[i:i + batch_size])
        if bool1:
            chunk = chunk.copy(deep = True)
            y_chunk = chunk['Type']
            y_chunk = y_chunk.to_numpy()
            y_chunk = y_chunk.reshape(y_chunk.shape[0],1)
            chunk.drop('Type', axis = 1, inplace = True)
            chunk = chunk.to_numpy()
            y_train.append(y_chunk)
        x_train.append(chunk)
    return x_train, y_train

In [1552]:
X_train, Y_train = get_batches(df,10,True)

Before implementing one hot and data normalization, I was getting an overflow on my softmax function due to the values being too large.
This would cause me to divide by zero when it became NaN.

Part 1:

In [1554]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.shape[1], 8))
    one_hot_Y[np.arange(Y.shape[1]), Y] = 1
    return one_hot_Y.T
class NeuralNetwork:
    def __init__(self):
        self.w1 = np.random.rand(64, 9) + .5
        self.b1 = np.random.rand(64,1) + .5
        self.w2 = np.random.rand(8,64) + .5
        self.b2 = np.random.rand(8,1) + .5
    def __str__(self):
        return
    def ReLU(self,Z):
        return np.maximum(Z,0)
    def d_reLU(self,Z):
        return Z > 0 
    def forward_propogation(self,X):
        Z1 = self.w1.dot(X) + self.b1
        A1 = self.ReLU(Z1)
        Z2 = self.w2.dot(A1) + self.b2
        A2 = self.softmax(Z2)
        return Z1,A1,Z2,A2
    def softmax(self,Z):
        A = np.exp(Z) / sum(np.exp(Z))
        return A

    def mse_loss(self, Y, Y_pred):
        return np.mean((Y - Y_pred) ** 2)
    def backward_propogation(self,A1,Z1,A2,Z2,X,Y):
        m = Y.shape[1]
        one_hot_Y = one_hot(Y)
        dZ2 = A2 - one_hot_Y
        dW2 = 1/m * np.dot(dZ2, A1.T) 
        dB2 = 1/m * np.sum(dZ2,keepdims= True)
        dZ1 = self.w2.T.dot(dZ2) * self.d_reLU(Z1)
        dW1 = 1/m * dZ1.dot(X.T)
        dB1 = 1/m * np.sum(dZ1,keepdims= True)
        return dW1,dW2,dB1,dB2
    def update_weights(self,lr,dW1,dW2,dB1,dB2):
        self.w1 -= lr * dW1
        self.b1 -= lr * dB1
        self.w2 -= lr * dW2
        self.b2 -= lr * dB2
        

    def predict(self, A2):
        return np.argmax(A2,0)
    def train(self,lr,X,Y):
        Z1,A1,Z2,A2 = self.forward_propogation(X)
        y_pred = self.predict(A2)
        loss = self.mse_loss(Y,y_pred)
        dW1, dW2, dB1, dB2 = self.backward_propogation(A1,Z1,A2,Z2,X,Y)
        self.update_weights(lr,dW1,dW2,dB1,dB2)
        return loss, y_pred
    

def accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size
    



In [1558]:
nn = NeuralNetwork()

In [1561]:
def min_max_normalize(x):
    minx = X.min(axis=0)
    xmax = X.max(axis=0)
    Xnorm = (x - minx) / (xmax - minx)
    return Xnorm


I decided to batch my data in order to increase efficency.

In [1559]:
for j in range(1000):
    total_acc = 0
    total_loss = 0
    for i in range(len(X_train)):
        loss, y_pred = nn.train(.1,min_max_normalize(X_train[i].T),Y_train[i].T)
        acc = (accuracy(y_pred, Y_train[i].T))
        total_acc += acc
        total_loss += loss
    print(f'Accuracy: {total_acc/len(X_train)}')
    print(f'Loss: {total_loss/len(X_train)}')

    

Accuracy: 0.7181818181818181
Loss: 1.8227272727272728
Accuracy: 0.2545454545454545
Loss: 18.96818181818182
Accuracy: 0.34545454545454546
Loss: 5.559090909090909
Accuracy: 0.34545454545454546
Loss: 5.559090909090909
Accuracy: 0.3909090909090909
Loss: 5.513636363636364
Accuracy: 0.3909090909090909
Loss: 5.513636363636364
Accuracy: 0.3
Loss: 5.6045454545454545
Accuracy: 0.36818181818181817
Loss: 6.172727272727273
Accuracy: 0.3
Loss: 5.6045454545454545
Accuracy: 0.3909090909090909
Loss: 4.422727272727273
Accuracy: 0.4227272727272727
Loss: 7.390909090909091
Accuracy: 0.3
Loss: 5.6045454545454545
Accuracy: 0.34545454545454546
Loss: 5.559090909090909
Accuracy: 0.4818181818181818
Loss: 7.490909090909091
Accuracy: 0.3409090909090909
Loss: 5.5636363636363635
Accuracy: 0.34545454545454546
Loss: 5.559090909090909
Accuracy: 0.34545454545454546
Loss: 5.559090909090909
Accuracy: 0.3909090909090909
Loss: 5.513636363636364
Accuracy: 0.3909090909090909
Loss: 5.513636363636364
Accuracy: 0.390909090909090

Part 2: 

Task 1: I needed the nn import in order to create my neural network and Data Loader in order to get my csv into a good formate for training. Many of the examples on PyTorch use this. The optim import is just for my optimizer

In [1067]:
import torch
import torchvision
from torch import nn as n
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.optim as optim
import copy

Task 2 from here below: Train and Test Datasets needed some work from just being numpy arrays. When I tried originally with them, I was running into issues with types. Doing it by converting to torch tensors with torch dtypes fixed a lot of my issues. I then changed them into to tensor datasets in order to have my values together. 

In [1582]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('glass.csv')
X = df.drop('Type', axis=1).values
Y = df['Type'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.long)

X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=True)

In [1589]:
class Net(n.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.seq = n.Sequential(n.Linear(9, 128)
        ,n.ReLU(),
        n.Linear(128, 128),
        n.ReLU(),
        n.Linear(128, 8))
    
    def forward(self, x):
        #The forward propogation. Im using relu as my activation function
        return self.seq(x)

In [1590]:
model = Net()

In [1591]:
lr = 0.1
optimizer = optim.Adam(model.parameters(),lr)
loss = n.CrossEntropyLoss()

In [1592]:
def train(model, loss_fn, optimizer, train_loader):
    model.train()
    total_loss = 0
    for X_batch, Y_batch in train_loader:
        # this was my gradient descent We would calculate the loss and thenn use it to do the backward propogation. We would calculate the total lost by the end
        optimizer.zero_grad()
        output = model(X_batch)
        loss = loss_fn(output, Y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f'Training loss: {avg_loss}')

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            output = model(X_batch)
            #This is equivalent to a softmax function to find the actual prediction
            _, predicted = torch.max(output.data, 1)
            total += Y_batch.size(0)
            correct += (predicted == Y_batch).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

In [1594]:
for epoch in range(1000):  #Running Train and Test
    print(f'Epoch {epoch+1}')
    train(model, loss, optimizer, train_loader)
    test(model, test_loader)

Epoch 1
Training loss: 1.4749383926391602
Accuracy: 32.55813953488372%
Epoch 2
Training loss: 1.470792060548609
Accuracy: 32.55813953488372%
Epoch 3
Training loss: 1.473633581941778
Accuracy: 32.55813953488372%
Epoch 4
Training loss: 1.4982598532329907
Accuracy: 25.58139534883721%
Epoch 5
Training loss: 1.4798301187428562
Accuracy: 32.55813953488372%
Epoch 6
Training loss: 1.501905769109726
Accuracy: 25.58139534883721%
Epoch 7
Training loss: 1.4774472117424011
Accuracy: 32.55813953488372%
Epoch 8
Training loss: 1.4710200862451033
Accuracy: 25.58139534883721%
Epoch 9
Training loss: 1.468632698059082
Accuracy: 32.55813953488372%
Epoch 10
Training loss: 1.477489021691409
Accuracy: 32.55813953488372%
Epoch 11
Training loss: 1.4955509034070102
Accuracy: 32.55813953488372%
Epoch 12
Training loss: 1.4771831252358176
Accuracy: 32.55813953488372%
Epoch 13
Training loss: 1.5017394856973127
Accuracy: 32.55813953488372%
Epoch 14
Training loss: 1.486597004261884
Accuracy: 25.58139534883721%
Epoch 1

Task 3: I selected a learning rate of .1 because this is pretty standard. I regularized my data in part 1 but in part 2 there was no need to since torch handled the values well. I used the Adam optimization algorithm. I used Adam to stablize the training process.